# Outline

1: First-class functions

2: Closures

3: Decorators

# First-class functions and Closures

Allows for functions to be passed as arguments to other functions, returned, and assigned to variables.  

### funtion variable assignment

In [2]:
def square (x):
    return x * x

f = square(5)

# create instance of square, can now 
#f = square

# refers to the function instance, did not invoke square
print (square)
print (f)
#print(f(5))

<function square at 0x10f3cfae8>
25


### passing functions as arguments

In [6]:
def my_map(func,my_list):
    results = []
    for i in my_list:
        results.append(func(i))
        
    return results

squares = my_map(square,[1,2,3,4])

print (squares)

[1, 4, 9, 16]


### functions that return functions

this brings us to the next topic - closures.  These are a type of first-class functions that include a nested function.  Below is a simple example

In [9]:
def logger (msg):
    
    def log_message():
        print('Log: ',msg)
    return log_message

log_hi=logger('Hi')
log_hi()

Log:  Hi


The novel thing about closures is that they remember the namespace of the encapsulating function

In [12]:
def html_tag(tag):
    
    def wrap_text(msg):
        print('<{0}>{1}<{0}>'.format(tag,msg))
    
    # notice no () being returned
    return wrap_text

# print_h1 now equals returned fucntion
print_h1 = html_tag('h1')
print(print_h1)
print_h1 ('Test Headline')
print_h1 ('Another Headline')

print_p = html_tag('p')
print_p('para stuff')

<function html_tag.<locals>.wrap_text at 0x10f588840>
<h1>Test Headline<h1>
<h1>Another Headline<h1>
<p>para stuff<p>


In [30]:
import logging

logging.basicConfig(filename='example.log',level=logging.INFO)

def logger(func):
    def log_func(*args):
        logging.info('Running {} with arguments {}'.format(func.__name__,args))
        print(func(*args))
    return log_func

def add(x,y):
    return x+y

def sub (x,y):
    return x-y

add_logger = logger(add)
sub_logger = logger(sub)

add_logger(3,4)
sub_logger(5,2)

7
3


You may be thinking to yourself, "that's useful but seems tedious" and you'd be corrent. Thankfully python provides us a more straight forward way to dynamically change functions. 

let's take examples from to the closers section and instead of returning a function we return a function call.  In this example we can change display by adding code into the decorator_function

In [34]:
def decorator_function(original_function):
    
    def wrapper_function():
        #print('executed before {}'.format(original_function.__name__))
        return original_function()
    
    return wrapper_function

def display():
    print('display function ran')
    
decorated_display = decorator_function(display)

decorated_display()

executed before display
display function ran


Those of you familiar with decoarators may be wondering where the '@' symbol is. The below code is functionally the same as above but uses a cleaner and more readable syntax.

In [43]:
def decorator_function(original_function):
  
    def wrapper_function(*args,**kwargs):
        print('executed before {}'.format(original_function.__name__))
        return original_function(*args,**kwargs)
    
    return wrapper_function

#can also create decorator classes
class decorator_class(object):
    def __init__(self,original_function):
        self.original_function = original_function
        
    def __call__(self,*args,**kwargs):
        print('executed before {}'.format(self.original_function.__name__))
        return self.original_function(*args,**kwargs)

@decorator_function
#@decorator_class
def display():
    print('display function ran')

display()

executed before display
display function ran


some practical examples

In [54]:
def my_timer(orig_func):
    import time
    
    def wrapper(*args,**kwargs):
        t1 = time.time()
        results = orig_func(*args,**kwargs)
        t2 = time.time()-t1
        print('{} ran in {} sec'.format(orig_func.__name__,t2))
        return results
    return wrapper

@my_timer
def display_info(name,age):
    print ('display_info ran with arguments {} and {}'.format(name,age))
    
display_info('scott',2)

display_info ran with arguments scott and 2
display_info ran in 5.5789947509765625e-05 sec


What if you wanted to use multiple decorators at once? You have to use a decorator on the wrapper functions inside of your decorator.

In [56]:
from functools import wraps

def my_timer(orig_func):
    import time
    
    @wraps(orig_func)
    def wrapper(*args,**kwargs):
        t1 = time.time()
        results = orig_func(*args,**kwargs)
        t2 = time.time()-t1
        print('{} ran in {} sec'.format(orig_func.__name__,t2))
        return results
    return wrapper

def my_logger(func,**kwargs):
    
    logging.basicConfig(filename='{}.log'.format(func.__name__),level=logging.INFO)
    
    @wraps(func)
    def wrapper(*args,**kwargs):
        logging.info('Running {} with arguments {}'.format(func.__name__,args))
        print(func(*args,**kwargs))
    return wrapper


@my_timer
@my_logger
def display_info(name,age):
    print ('display_info ran with arguments {} and {}'.format(name,age))
    
display_info('scott',2)

display_info ran with arguments scott and 2
None
display_info ran in 0.0002617835998535156 sec


You can also create decorators that accept arguments

In [57]:
def prefix_decorator(prefix):
    def decorator_function(original_function):
        def wrapper_function(*args,**kwargs):
            print(prefix,'Executed before',original_function.__name__)
            result=original_function(*args,**kwargs)
            print(prefix,'Executed after',original_function.__name__, '\n')
            return result
        return wrapper_function
    return decorator_function

@prefix_decorator('PREF: ')
def display_info(name,age):
    print('display_info ran with {} {}'.format(name,age))
    
display_info('scott',8)

PREF:  Executed before display_info
display_info ran with scott 8
PREF:  Executed after display_info 



# Static/Class/Property decorators

Those familiar with python classes may have come accross decorators used.  Stat methods and class methods are similar but a little out of scope for this guide.  They should be covered during some OOP tutorial.


property decorators allow us to access methods as if they were attributes

In [65]:
class Employee():
    def __init__(self,first,last):
        self.first = first
        self.last = last
        
    @property
    def email(self):
        return ('{}.{}@company.com'.format(self.first,self.last))
    
    def full_name(self):
        return ("{} {}".format(self.first,self.last))
    
emp_1 = Employee('Scott','Christ')

print(emp_1.email)

Scott.Christ@company.com


If the property decorator allows you to access methods as attributes, then you should be able to change their values aswell.  The setter decorator accomplishes this.

In [71]:
class Employee():
    def __init__(self,first,last):
        self.first = first
        self.last = last
        
    @property
    def email(self):
        return ('{}.{}@company.com'.format(self.first,self.last))
    
    @property
    def full_name(self):
        return ("{} {}".format(self.first,self.last))
    
    @full_name.setter
    def full_name(self,name):
        first, last = name.split(' ')
        self.first = first
        self.last = last

emp_2 = Employee('Charles','Christ')
emp_2.full_name = 'Scott Christ'

print(emp_2.first)

Scott
